In [2]:
from amalur.AmalurMatrix import AmalurMatrix
import pandas as pd
import numpy as np

S1 = np.array([[55],[76]])
S2 = np.array([[0,1,0,0,0.005],
               [1, 0, 0, 0, 0.4],
               [0, 1, 0, 0, 0.01]])
S3 = np.array([[55,171]])

M1 = np.array([[1], [0], [0], [0], [0], [0], [0]])
M2 = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
])
M3 = np.array([
    [1, 0],
    [0, 1],
    [0, 0],
    [0, 0],
    [0, 0],
    [0, 0],
    [0, 0]
])

I1 = np.array([
    [1, 0],
    [0, 1],
    [0, 1]
])

I2 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

I3 = np.array([
    [1],
    [0],
    [0]
])

2023-12-24 13:14:05.458 | INFO     | amalur.AmalurMatrix:<module>:34 - Loaded AmalurMatrix with USE_GPU=False & USE_MKL=True


## Tables
<table>
<tr><th>S1 </th><th>S2</th><th>S3</th></tr>
<tr><td>

|   |
|---|
| 55|
| 76|

</td><td>

|   |   |   |   |      |
|---|---|---|---|------|
| 0 | 1 | 0 | 0 | 0.005|
| 1 | 0 | 0 | 0 | 0.4  |
| 0 | 1 | 0 | 0 | 0.01 |


</td><td>

|   |   |
|---|---|
| 55|171|
</td></tr> </table>



<table>
<tr><th>M1 </th><th>M2</th><th>M3</th></tr>
<tr><td>

|   |   
|---|
| 1 |
| 0 |
| 0 |
| 0 |
| 0 |
| 0 |
| 0 |

</td><td>

|   |   |   |   |   |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 0 | 0 | 1 |


</td><td>

|   |   |
|---|---|
| 1 | 0 |
| 0 | 1 |
| 0 | 0 |
| 0 | 0 |
| 0 | 0 |
| 0 | 0 |
| 0 | 0 |

</td></tr> </table>


<table>
<tr><th>I1 </th><th>I2</th><th>I3</th></tr>
<tr><td>

|   |   |
|---|---|
| **1** | 0 |
| 0 | 1 |
| 0 | 1 |

</td><td>

|   |   |   |
|---|---|---|
| 1 | 0 | 0 |
| 0 | 1 | 0 |
| 0 | 0 | 1 |


</td><td>

|   |
|---|
| **1** | 
| 0 |
| 0 |
</td></tr> </table>

In [24]:
s1 = pd.DataFrame(S1, columns=['weight'])
s1.index=['Olivia', 'Bart']

s2 = pd.DataFrame(S2, columns=[*[f'medicine_{x}' for x in range(4)], 'dosage'])
s2.index=['Olivia', 'Bart', 'Bart']

s3 = pd.DataFrame(S3, columns=['weight', 'height'])
s3.index=['Olivia']

T = pd.merge(pd.merge(s1, s2, left_index=True, right_index=True), s3, left_index=True, right_index=True, how='left')
T = T[['weight_x', 'height', 'medicine_0', 'medicine_1', 'medicine_2', 'medicine_3', 'dosage']]
T.sort_values(by=['weight_x'], ascending=True)

,weight_x,height,medicine_0,medicine_1,medicine_2,medicine_3,dosage
Olivia,55,171.0,0.0,1.0,0.0,0.0,0.005
Bart,76,NaN,1.0,0.0,0.0,0.0,0.400
Bart,76,NaN,0.0,1.0,0.0,0.0,0.010


Current way described in paper results in Olivia's weight being doubled.

In [62]:
S, I, M = [S1, S2, S3], [I1, I2, I3], [M1, M2, M3]
amalur = AmalurMatrix(S, I, M)
pd.DataFrame(amalur.materialize(amalur))

,0,1,2,3,4,5,6
0,110.0,171.0,0.0,1.0,0.0,0.0,0.005
1,76.0,0.0,1.0,0.0,0.0,0.0,0.400
2,76.0,0.0,0.0,1.0,0.0,0.0,0.010


In [26]:
new_i1 = np.array([
    [0, 0],
    [0, 1],
    [0, 1]
])
amalur_new = AmalurMatrix([S1, S2, S3], [new_i1, I2, I3], [M1, M2, M3])
pd.DataFrame(amalur_new.materialize(amalur_new))

,0,1,2,3,4,5,6
0,55.0,171.0,0.0,1.0,0.0,0.0,0.005
1,76.0,0.0,1.0,0.0,0.0,0.0,0.400
2,76.0,0.0,0.0,1.0,0.0,0.0,0.010


Only include values for which there is no value in T yet.

In [82]:
from amalur.AmalurMatrix import multi_dot
result = multi_dot([I[0], S[0], M[0].T])
for k in range(1, 3):
    m = multi_dot([I[k], S[k], M[k].T])
    mask = (result == 0).astype(int)
    m *= mask
    result += m
pd.DataFrame(result)

,0,1,2,3,4,5,6
0,55.0,171.0,0.0,1.0,0.0,0.0,0.005
1,76.0,0.0,1.0,0.0,0.0,0.0,0.400
2,76.0,0.0,0.0,1.0,0.0,0.0,0.010
